In [ ]:
%load_ext autoreload
%autoreload 2

import logging
import os

import core.config_builders as ccbuild
import core.dataflow as cdataf
import helpers.dbg as dbg
import helpers.env as henv
import helpers.pickle_ as hpickl
import helpers.printing as hprint

In [ ]:
dbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

In [ ]:
config = ccbuild.get_config_from_env()

In [ ]:
dag_config = config.pop("DAG")

In [ ]:
dag_runner = cdataf.PredictionDagRunner(dag_config, config["meta"]["dag_builder"])

In [ ]:
cdataf.draw(dag_runner.dag)

In [ ]:
if "set_fit_intervals" in config["meta"].to_dict():
    dag_runner.set_fit_intervals(
        **config["meta", "set_fit_intervals", "func_kwargs"].to_dict()
    )
if "set_predict_intervals" in config["meta"].to_dict():
    dag_runner.set_predict_intervals(
        **config["meta", "set_predict_intervals", "func_kwargs"].to_dict()
    )

In [ ]:
fit_result_bundle = dag_runner.fit()

In [ ]:
payload = ccbuild.get_config_from_nested_dict({"config": config})

In [ ]:
if "run_oos" in config["meta"].to_dict().keys() and config["meta"]:
    result_bundle = dag_runner.predict()
    payload["fit_result_bundle"] = fit_result_bundle.to_config()
else:
    result_bundle = fit_result_bundle

In [ ]:
result_bundle.payload = payload

In [ ]:
try:
    path = os.path.join(
        config["meta", "experiment_result_dir"], "result_bundle.pkl"
    )
    if True:
        hpickl.to_pickle(result_bundle.to_config().to_dict(), path)
except AssertionError:
    _LOG.warning("Unable to serialize results.")